In [58]:
import os
import sys

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F  
import matplotlib 
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import torch.utils.data
from torch.utils.data import DataLoader


In [22]:
sys.path.append("/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/models") #probably keep this still
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/best_model.pth"
IQ_FILE_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m/Pluto_0_2m_run0.iq"
#MODEL_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/justin_model_slicing_norm_LR1_smallHop_12_largerInput.pth"
#IQ_FILE_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m/Pluto_1_1m_run0.iq"

In [ ]:
from attempt2 import resnet50_1d  # Directly import from attempt2.py
num_classes = 8  # Change this if your model was trained with a different number of classes

# Initialize the model architecture
model = resnet50_1d(num_classes=num_classes).to(DEVICE)

# Load trained weights
print(f"Loading trained model weights from: {MODEL_PATH}")
state_dict = torch.load(MODEL_PATH, map_location=DEVICE)

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()

In [24]:
def load_iq_data(file_path, max_samples=4096*2, start_idx=0):
    total_samples = max_samples * 2  # Since I/Q samples are interleaved

    #  Open the file in binary mode and seek to `start_idx`
    with open(file_path, "rb") as f:
        f.seek(start_idx * 4 * 2)  # 4 bytes per float32, 2 channels (I/Q)
        raw_data = np.fromfile(f, dtype="float32", count=total_samples)

    #  Ensure we have enough data
    if raw_data.shape[0] < total_samples:
        raise ValueError(f"Not enough data in {file_path}. Requested {total_samples}, got {raw_data.shape[0]}.")

    #  Extract I/Q channels
    I = raw_data[0::2]  # Even indices
    Q = raw_data[1::2]  # Odd indices

    #  Stack into [2, max_samples] format
    iq_data = np.stack([I, Q], axis=0)

    #  Add batch dimension → [1, 2, max_samples]
    iq_data = np.expand_dims(iq_data, axis=0)

    #  Convert to PyTorch tensor
    data_tensor = torch.from_numpy(iq_data).float()

    #  Dummy label for now (adjust if needed)
    label_tensor = torch.tensor([0], dtype=torch.long)

    return data_tensor, label_tensor

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

def main():
    print(f"Loading data from: {IQ_FILE_PATH}")
    data, labels = load_iq_data(IQ_FILE_PATH)
    data, labels = data.to(DEVICE), labels.to(DEVICE)

    model.eval()
    
    with torch.no_grad():
        logits = model(data)  # Raw model output
        probs = F.softmax(logits, dim=1)  # Convert to probabilities
        original_pred = torch.argmax(probs, dim=1)  # Get predicted class
        confidence = probs.max(dim=1).values  # Get confidence score of predicted class

    print(f"Original prediction: {original_pred.cpu().numpy()}, Confidence: {confidence.cpu().item():.4f}")

if __name__ == "__main__":
    main()

Loading data from: /home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m/Pluto_0_2m_run0.iq
Original prediction: [5], Confidence: 0.9728


In [ ]:
IQ_DIR_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m"
def main():
    print(f"Loading data from directory: {IQ_DIR_PATH}")
    iq_files = [f for f in os.listdir(IQ_DIR_PATH) if f.endswith(".iq")]

    for fname in sorted(iq_files):
        file_path = os.path.join(IQ_DIR_PATH, fname)
        #print(f"\nProcessing file: {file_path}")
        
        data, labels = load_iq_data(file_path)
        data, labels = data.to(DEVICE), labels.to(DEVICE)

        model.eval()
        with torch.no_grad():
            logits = model(data)
            probs = F.softmax(logits, dim=1)
            original_pred = torch.argmax(probs, dim=1)
            confidence = probs.max(dim=1).values

        #print(f"Prediction: {original_pred.cpu().numpy()}, Confidence: {confidence.cpu().numpy()}")

if __name__ == "__main__":
    main()

In [47]:
class IQDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        sample = torch.from_numpy(sample).float()
        # Normalize data
        magnitude = torch.sqrt(torch.sum(sample**2, dim=1, keepdim=True))
        sample = sample / magnitude

        label_tensors = torch.tensor(label, dtype=torch.long)

        return sample, label_tensors

In [63]:
IQ_DIR_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m"
BATCH_SIZE = 16
WINDOW_SIZE = 10000
HOP_SIZE = 500
START_INDEX = 4800
END_INDEX = 6000
TARGET_LABEL = 0

# Initialize and load model BEFORE the main loop
model = resnet50_1d(num_classes=8)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

def main():
    print(f"Loading data from directory: {IQ_DIR_PATH}")
    iq_files = [f for f in os.listdir(IQ_DIR_PATH) if f.endswith(".iq")]

    for fname in sorted(iq_files):
        file_path = os.path.join(IQ_DIR_PATH, fname)
        print(f"\nProcessing file: {file_path}")

        # Extract label from filename: Pluto_#_...
        try:
            target_label = int(fname.split("_")[1])
        except (IndexError, ValueError):
            print(f"  Warning: could not parse label from filename: {fname}")
            continue

        # Load IQ data
        data = np.fromfile(file_path, dtype="float32")
        real_part = data[0::2]
        imag_part = data[1::2]

        test_data_tensors = []
        test_label_tensors = []

        # Sliding window generation
        for x in range(START_INDEX, END_INDEX):
            start = (x + 1) * HOP_SIZE
            end = start + WINDOW_SIZE
            if end > len(real_part): break

            i_window = real_part[start:end]
            q_window = imag_part[start:end]
            combined = np.vstack((i_window, q_window))  # [2, WINDOW_SIZE]
            test_data_tensors.append(combined)
            test_label_tensors.append(target_label)

        if not test_data_tensors:
            print(f"Skipping {fname}: not enough valid IQ segments.")
            continue

        # Stack and shuffle
        test_data_tensors = np.stack(test_data_tensors, axis=0)
        test_label_tensors = np.array(test_label_tensors)
        indices = np.random.permutation(len(test_data_tensors))
        test_data = test_data_tensors[indices]
        test_labels = test_label_tensors[indices]

        # Create Dataset + Loader
        test_dataset = IQDataset(test_data, test_labels)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

        correct = 0
        total = 0
        mismatch_count = 0
        with torch.no_grad():
            for data in test_loader:
                images, labels = data
                images = images.to(DEVICE)
                labels = labels.to(DEVICE)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                mismatch_count += (predicted != labels).sum().item()
                correct += (predicted == labels).sum().item()
        
        acc = correct / total * 100
        print(f"[{fname}] Accuracy: {acc:.2f}% | Mismatches: {mismatch_count} / {total}")
        



if __name__ == "__main__":
    main()

/tmp/ipykernel_3723382/3204918950.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))


Loading data from directory: /home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m

Processing file: /home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m/Pluto_0_1m_run0.iq
[Pluto_0_1m_run0.iq] Accuracy: 76.75% | Mismatches: 279 / 1200

Processing file: /home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m/Pluto_0_1m_run1.iq
[Pluto_0_1m_run1.iq] Accuracy: 98.17% | Mismatches: 22 / 1200

Processing file: /home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m/Pluto_0_2m_run0.iq
[Pluto_0_2m_run0.iq] Accuracy: 98.75% | Mismatches: 15 / 1200

Processing file: /home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m/Pluto_0_2m_run1.iq
[Pluto_0_2m_run1.iq] Accuracy: 88.33% | Mismatches: 140 / 1200

Processing file: /home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m/Pluto_1_1m_run0.iq
[Pluto_1_1m_run0.iq] Accuracy: 100.00% | Mismatches: 0 / 1200

Processing file: /home/jfeng/